In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd

# use seaborn plotting defaults
import seaborn as sns; sns.set()

In [3]:
sdata = {"Ohio": 35000, "Texas": 71000, "Oregon": 16000, "Utah": 5000}

obj3 = pd.Series(sdata)
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [4]:
obj3.to_dict()

{'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}

In [9]:
obj3.isna()

Ohio      False
Texas     False
Oregon    False
Utah      False
dtype: bool

In [38]:
data = {"state": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
        "year": [2000, 2001, 2002, 2001, 2002, 2003],
        "pop": [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)

In [19]:
frame.state.name = 'stato'
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


In [21]:
pd.DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


In [26]:
frame['debt'] = np.arange(6.0)
frame

,state,year,pop,debt
0,Ohio,2000,1.5,0.0
1,Ohio,2001,1.7,1.0
2,Ohio,2002,3.6,2.0
3,Nevada,2001,2.4,3.0
4,Nevada,2002,2.9,4.0
5,Nevada,2003,3.2,5.0


In [28]:
val = pd.Series([-1.2, -1.5, -1.7], index=[2, 4, 5])
frame.iloc[val.index.to_list(),3] = val
frame

,state,year,pop,debt
0,Ohio,2000,1.5,0.0
1,Ohio,2001,1.7,1.0
2,Ohio,2002,3.6,-1.2
3,Nevada,2001,2.4,3.0
4,Nevada,2002,2.9,-1.5
5,Nevada,2003,3.2,-1.7


In [29]:
frame.index.name = 'peppa'
frame.columns.name = 'pig'
frame

pig,state,year,pop,debt
peppa,,,,
0,Ohio,2000,1.5,0.0
1,Ohio,2001,1.7,1.0
2,Ohio,2002,3.6,-1.2
3,Nevada,2001,2.4,3.0
4,Nevada,2002,2.9,-1.5
5,Nevada,2003,3.2,-1.7


In [41]:
frame2 = frame
frame2

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


In [49]:
frame2 = frame2.reindex(list(range(6, -1, -1)))
frame2

,state,year,pop
6,NaN,NaN,NaN
5,Nevada,2003.0,3.2
4,Nevada,2002.0,2.9
3,Nevada,2001.0,2.4
2,Ohio,2002.0,3.6
1,Ohio,2001.0,1.7
0,Ohio,2000.0,1.5


In [48]:
list(range(6, -1, -1))

[6, 5, 4, 3, 2, 1, 0]